# Chapter 7

## Aggregations

In [4]:
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("gs://reddys-data-for-experimenting/retail-data/all/*.csv") \
    .coalesce(5)

In [5]:
df.count()

541909

In [6]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [7]:
df.show(5)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
only showing top 5 rows



In [9]:
from pyspark.sql.functions import count

df.select(count("StockCode")).show()

+----------------+
|count(StockCode)|
+----------------+
|          541909|
+----------------+



In [10]:
from pyspark.sql.functions import countDistinct
df.select(countDistinct("StockCode")).show()

+-------------------------+
|count(DISTINCT StockCode)|
+-------------------------+
|                     4070|
+-------------------------+



In [12]:
# Use this when you an exact number is not needed
from pyspark.sql.functions import approx_count_distinct

df.select(approx_count_distinct("StockCode")).show()

+--------------------------------+
|approx_count_distinct(StockCode)|
+--------------------------------+
|                            3804|
+--------------------------------+



In [13]:
from pyspark.sql.functions import first, last

df.select(first("StockCode"), last("StockCode")).show()

+-----------------------+----------------------+
|first(StockCode, false)|last(StockCode, false)|
+-----------------------+----------------------+
|                  21544|                85049D|
+-----------------------+----------------------+



In [15]:
from pyspark.sql.functions import min, max

df.select(min("Quantity"), max("Quantity")).show()

+-------------+-------------+
|min(Quantity)|max(Quantity)|
+-------------+-------------+
|       -80995|        80995|
+-------------+-------------+



In [16]:
from pyspark.sql.functions import sum, sumDistinct

df.select(sum("Quantity"), sumDistinct("Quantity")).show()

+-------------+----------------------+
|sum(Quantity)|sum(DISTINCT Quantity)|
+-------------+----------------------+
|      5176450|                 29310|
+-------------+----------------------+



In [21]:
from pyspark.sql.functions import sum, count, avg, expr, mean

df.select(
    count("Quantity").alias("total_transactions"),
    sum("Quantity").alias("total_quantity"),
    avg("Quantity").alias("avg_quantity"),
    mean("Quantity").alias("mean_quantity")
).selectExpr(
    "total_quantity/total_transactions as manual_avg",
    "avg_quantity",
    "mean_quantity"
).show()

+----------------+----------------+----------------+
|      manual_avg|    avg_quantity|   mean_quantity|
+----------------+----------------+----------------+
|9.55224954743324|9.55224954743324|9.55224954743324|
+----------------+----------------+----------------+



### Other function can be used in the same way like

* skewness
* kurtosis
* var_pop
* stddev_pop
* var_samp
* stddev_samp
* corr
* covar_pop
* covar_samp
* collect_set
* collect_list

## Grouping

In [24]:
from pyspark.sql.functions import count, expr

df.groupBy("InvoiceNo") \
    .agg(count("StockCode").alias("throughFunction"),
         expr("count(StockCode)").alias("ThroughExpression")
        ).show(5)

+---------+---------------+-----------------+
|InvoiceNo|throughFunction|ThroughExpression|
+---------+---------------+-----------------+
|   536596|              6|                6|
|   536938|             14|               14|
|   537252|              1|                1|
|   537691|             20|               20|
|   538041|              1|                1|
+---------+---------------+-----------------+
only showing top 5 rows



### Rollups

In [26]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [28]:
rolledUpDF = df.rollup("InvoiceDate", "Country").agg(sum("Quantity"))\
  .selectExpr("InvoiceDate", "Country", "`sum(Quantity)` as total_quantity")\
  .orderBy("InvoiceDate")
rolledUpDF.show(10)

+---------------+--------------+--------------+
|    InvoiceDate|       Country|total_quantity|
+---------------+--------------+--------------+
|           null|          null|       5176450|
|1/10/2011 10:04|United Kingdom|           -29|
|1/10/2011 10:04|          null|           -29|
|1/10/2011 10:07|          EIRE|            -4|
|1/10/2011 10:07|          null|            -4|
|1/10/2011 10:08|          null|           -14|
|1/10/2011 10:08|United Kingdom|           -14|
|1/10/2011 10:32|United Kingdom|           260|
|1/10/2011 10:32|          null|           260|
|1/10/2011 10:35|          null|           408|
+---------------+--------------+--------------+
only showing top 10 rows



In [30]:
from pyspark.sql.functions import col

df.cube("InvoiceDate", "Country").agg(sum(col("Quantity")))\
  .select("InvoiceDate", "Country", "sum(Quantity)").orderBy("InvoiceDate").show()

+-----------+--------------------+-------------+
|InvoiceDate|             Country|sum(Quantity)|
+-----------+--------------------+-------------+
|       null|               Japan|        25218|
|       null|           Australia|        83653|
|       null|            Portugal|        16180|
|       null|             Germany|       117448|
|       null|                null|      5176450|
|       null|United Arab Emirates|          982|
|       null|                 RSA|          352|
|       null|             Finland|        10666|
|       null|             Lebanon|          386|
|       null|         Unspecified|         3300|
|       null|              Cyprus|         6317|
|       null|      Czech Republic|          592|
|       null|              Norway|        19247|
|       null|     Channel Islands|         9479|
|       null|           Hong Kong|         4769|
|       null|                 USA|         1034|
|       null|           Singapore|         5234|
|       null|       

In [38]:
spark.stop()